## Importações

In [1]:
import os
from itertools import product

import numpy as np
import pandas as pd
from scipy.stats import zscore

from sklearn.datasets import make_regression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.feature_selection import SelectKBest, f_regression, SelectPercentile
import pandas as pd
import numpy as np
from itertools import product
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split, KFold, RepeatedKFold, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.feature_selection import SelectKBest, f_regression, RFE
from scipy.stats import zscore
from sklearn.model_selection import cross_validate
import time


In [2]:
%pip install XlsxWriter

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Plano de Experimentação

### 1. Carregar o dataset

In [3]:
df = pd.read_csv('../../../data/pre-processado/por-municipio/completo/dados-manaus-preprocessado.csv') 

# Retirando antes dos anos 2000 pois dados de clima ausentes, e inserir pela média prejudicou o modelo
df = df[df['ano']>=2000]

# Retirando pois variáveis não necessárias para o modelo, visto que são iguais para todas as linhas
df = df.drop(columns=['municipio','uf'])

### 2. Divisão do Dataset

In [4]:
X = df.drop(columns='vazao')
y = df['vazao'] # Separa o target
cat_cols = X.select_dtypes(include='object').columns.tolist()

### 3. Pré-processamentos

In [5]:
def remover_outliers(df, colunas, threshold=3):
    z_scores = np.abs(zscore(df[colunas]))
    mask = (z_scores < threshold).all(axis=1)
    return df[mask]

### 4. Parâmetros de variação

In [6]:
classificacoes = ['2 classificações', '3 classificações']
validacao = ['k=3', 'k=10']
criterios = ['squared_error']
n_arvores = [50, 100]
etas = [0.1, 0.3]
depths = [3, 5]
normalizadores = [StandardScaler(), MinMaxScaler(), RobustScaler()]
usar_outliers = [True, False]
seletores = [None, SelectKBest(score_func=f_regression, k=5), RFE(estimator=RandomForestRegressor(), n_features_to_select=5)]
validadores = [('k=3', KFold(n_splits=3)), ('k=10', KFold(n_splits=10)), ('rep=5x2', RepeatedKFold(n_splits=5, n_repeats=2))]


### 5. Cenários

In [7]:
cenarios_rfr = list(product(classificacoes, criterios, n_arvores, normalizadores, usar_outliers, seletores, validadores))
cenarios_xgb = list(product(classificacoes, etas, depths, normalizadores, usar_outliers, seletores, validadores))

### 4. Random Forest Regressor

In [8]:
resultados_rfr = []

for i, (classe, criterio, n, norm, outlier_flag, seletor, (val_tag, validador)) in enumerate(cenarios_rfr):
    inicio = time.time()
    X_temp = X.copy()
    y_temp = y.copy()
    num_cols = X_temp.select_dtypes(include=['int64', 'float64']).columns.tolist()

    if outlier_flag:
        df_completo = X_temp.copy()
        df_completo["target"] = y_temp
        df_sem_outliers = remover_outliers(df_completo, num_cols)
        X_temp = df_sem_outliers.drop(columns="target")
        y_temp = df_sem_outliers["target"]

    if X_temp.shape[0] < 2:
        continue

    preprocessador = ColumnTransformer([("scale", norm, num_cols)], remainder='passthrough')
    etapas = [("prep", preprocessador)]

    if seletor:
        etapas.append(("sel", seletor))

    etapas.append(("clf", RandomForestRegressor(n_estimators=n, criterion=criterio, random_state=42, n_jobs=-1)))
    modelo = Pipeline(etapas)

    scores = cross_validate(modelo, X_temp, y_temp, cv=validador, scoring=[
            'neg_mean_squared_error', 'neg_mean_absolute_error', 'r2'
        ], n_jobs=-1)    
    mse = -np.mean(scores['test_neg_mean_squared_error'])
    mae = -np.mean(scores['test_neg_mean_absolute_error'])
    r2 = np.mean(scores['test_r2'])

    resultados_rfr.append([
        i + 1,
        'X' if classe == '2 classificações' else '',
        'X' if classe == '3 classificações' else '',
        criterio,
        n,
        'X' if val_tag == 'k=3' else '',
        'X' if val_tag == 'k=10' else '',
        'X' if outlier_flag else '',
        type(norm).__name__.replace('Scaler', ''),
        type(seletor).__name__ if seletor else "None",
        val_tag,
        mse,
        mae, 
        r2
    ])

    fim = time.time()
    print(f"Cenário {i+1}/{len(cenarios_rfr)} levou {fim - inicio:.2f} segundos")

Cenário 1/216 levou 4.69 segundos
Cenário 2/216 levou 10.33 segundos
Cenário 3/216 levou 6.87 segundos
Cenário 4/216 levou 0.94 segundos
Cenário 5/216 levou 2.72 segundos
Cenário 6/216 levou 2.84 segundos
Cenário 7/216 levou 116.44 segundos
Cenário 8/216 levou 270.13 segundos
Cenário 9/216 levou 205.13 segundos
Cenário 10/216 levou 2.61 segundos
Cenário 11/216 levou 9.63 segundos
Cenário 12/216 levou 7.84 segundos
Cenário 13/216 levou 0.72 segundos
Cenário 14/216 levou 2.58 segundos
Cenário 15/216 levou 2.58 segundos
Cenário 16/216 levou 86.60 segundos
Cenário 17/216 levou 217.74 segundos
Cenário 18/216 levou 184.55 segundos
Cenário 19/216 levou 2.52 segundos
Cenário 20/216 levou 10.46 segundos
Cenário 21/216 levou 9.29 segundos
Cenário 22/216 levou 0.72 segundos
Cenário 23/216 levou 3.12 segundos
Cenário 24/216 levou 2.73 segundos
Cenário 25/216 levou 101.61 segundos
Cenário 26/216 levou 201.76 segundos
Cenário 27/216 levou 153.92 segundos
Cenário 28/216 levou 2.66 segundos
Cenário 29

### 5. XGBoost Regressor

In [11]:
resultados_xgbr = []

for i, (classe, eta, depth, norm, outlier_flag, seletor, (val_tag, validador)) in enumerate(cenarios_xgb):
    inicio = time.time()
    X_temp = X.copy()
    y_temp = y.copy()
    num_cols = X_temp.select_dtypes(include=['int64', 'float64']).columns.tolist()

    if outlier_flag:
        df_completo = X_temp.copy()
        df_completo["target"] = y_temp
        df_sem_outliers = remover_outliers(df_completo, num_cols)
        X_temp = df_sem_outliers.drop(columns="target")
        y_temp = df_sem_outliers["target"]

    if X_temp.shape[0] < 2:
        continue

    preprocessador = ColumnTransformer([("scale", norm, num_cols)], remainder='passthrough')
    etapas = [("prep", preprocessador)]

    if seletor:
        etapas.append(("sel", seletor))

    etapas.append(("clf", XGBRegressor(learning_rate=eta, max_depth=depth, random_state=42,n_jobs=-1)))
    modelo = Pipeline(etapas)

    scores = cross_validate(modelo, X_temp, y_temp, cv=validador, scoring=[
            'neg_mean_squared_error', 'neg_mean_absolute_error', 'r2'
        ], n_jobs=-1)    
    mse = -np.mean(scores['test_neg_mean_squared_error'])
    mae = -np.mean(scores['test_neg_mean_absolute_error'])
    r2 = np.mean(scores['test_r2'])

    resultados_xgbr.append([
        i + 1,
        'X' if classe == '2 classificações' else '',
        'X' if classe == '3 classificações' else '',
        eta,
        depth,
        'X' if val_tag == 'k=3' else '',
        'X' if val_tag == 'k=10' else '',
        'X' if outlier_flag else '',
        type(norm).__name__.replace('Scaler', ''),
        type(seletor).__name__ if seletor else "None",
        val_tag,
        mse,
        mae,
        r2
    ])

    fim = time.time()
    print(f"Cenário {i+1}/{len(cenarios_xgb)} levou {fim - inicio:.2f} segundos")

Cenário 1/216 levou 3.67 segundos
Cenário 2/216 levou 4.05 segundos
Cenário 3/216 levou 0.58 segundos
Cenário 4/216 levou 0.39 segundos
Cenário 5/216 levou 0.52 segundos
Cenário 6/216 levou 0.28 segundos
Cenário 7/216 levou 120.79 segundos
Cenário 8/216 levou 258.47 segundos
Cenário 9/216 levou 161.55 segundos
Cenário 10/216 levou 0.26 segundos
Cenário 11/216 levou 0.45 segundos
Cenário 12/216 levou 0.43 segundos
Cenário 13/216 levou 0.20 segundos
Cenário 14/216 levou 0.29 segundos
Cenário 15/216 levou 0.27 segundos
Cenário 16/216 levou 105.20 segundos
Cenário 17/216 levou 230.39 segundos
Cenário 18/216 levou 182.59 segundos
Cenário 19/216 levou 0.29 segundos
Cenário 20/216 levou 0.42 segundos
Cenário 21/216 levou 0.46 segundos
Cenário 22/216 levou 0.23 segundos
Cenário 23/216 levou 0.33 segundos
Cenário 24/216 levou 0.32 segundos
Cenário 25/216 levou 88.45 segundos
Cenário 26/216 levou 176.32 segundos
Cenário 27/216 levou 165.62 segundos
Cenário 28/216 levou 0.27 segundos
Cenário 29/2

### 7. Dataframe com Resultados

In [28]:
df_rfr = pd.DataFrame(resultados_rfr.copy(), columns=[
    'Cenário', '2 classificações', '3 classificações', 'Critério',
    'Nº Árvores', 'Validação k=3', 'Validação k=10',
    'Remoção de Outliers', 'Normalização', 'Seleção de Atributos',
    'Validação Cruzada', 'MSE', 'MAE', 'R²'
])

df_xgbr = pd.DataFrame(resultados_xgbr.copy(), columns=[
    'Cenário', '2 classificações', '3 classificações', 'Taxa de Aprendizado',
    'Profundidade', 'Validação k=3', 'Validação k=10',
    'Remoção de Outliers', 'Normalização', 'Seleção de Atributos',
    'Validação Cruzada', 'MSE', 'MAE', 'R²'
])

### 7. Formatação do Dataframe

In [29]:
# XGBoost
df_xgbr['Taxa 0.1'] = df_xgbr['Taxa de Aprendizado'].apply(lambda x: 'X' if x == 0.1 else '')
df_xgbr['Taxa 0.3'] = df_xgbr['Taxa de Aprendizado'].apply(lambda x: 'X' if x == 0.3 else '')
df_xgbr['Profundidade 3'] = df_xgbr['Profundidade'].apply(lambda x: 'X' if x == 3 else '')
df_xgbr['Profundidade 5'] = df_xgbr['Profundidade'].apply(lambda x: 'X' if x == 5 else '')
df_xgbr['Standard'] = df_xgbr['Normalização'].apply(lambda x: 'X' if x == 'Standard' else '')
df_xgbr['MinMax'] = df_xgbr['Normalização'].apply(lambda x: 'X' if x == 'MinMax' else '')
df_xgbr['Robust'] = df_xgbr['Normalização'].apply(lambda x: 'X' if x == 'Robust' else '')
#-- Seleção de Atributos
df_xgbr['-'] = df_xgbr['Seleção de Atributos'].apply(lambda x: 'X' if x == 'None' else '')
df_xgbr['SelectKBest'] = df_xgbr['Seleção de Atributos'].apply(lambda x: 'X' if x == 'SelectKBest' else '')
df_xgbr['RFE'] = df_xgbr['Seleção de Atributos'].apply(lambda x: 'X' if x == 'RFE' else '')
#-- Validação Cruzada
df_xgbr['KFold k=3'] = df_xgbr['Validação Cruzada'].apply(lambda x: 'X' if x == 'k=3' else '')
df_xgbr['KFold k=10'] = df_xgbr['Validação Cruzada'].apply(lambda x: 'X' if x == 'k=10' else '')
df_xgbr['RepeatedKFold 5x2'] = df_xgbr['Validação Cruzada'].apply(lambda x: 'X' if x == 'rep=5x2' else '')

# Random Forest
df_rfr['Standard'] = df_rfr['Normalização'].apply(lambda x: 'X' if x == 'Standard' else '')
df_rfr['MinMax'] = df_rfr['Normalização'].apply(lambda x: 'X' if x == 'MinMax' else '')
df_rfr['Robust'] = df_rfr['Normalização'].apply(lambda x: 'X' if x == 'Robust' else '')
#-- Seleção de Atributos
df_rfr['-'] = df_rfr['Seleção de Atributos'].apply(lambda x: 'X' if x == 'None' else '')
df_rfr['SelectKBest'] = df_rfr['Seleção de Atributos'].apply(lambda x: 'X' if x == 'SelectKBest' else '')
df_rfr['RFE'] = df_rfr['Seleção de Atributos'].apply(lambda x: 'X' if x == 'RFE' else '')
#-- Validação Cruzada
df_rfr['KFold k=3'] = df_rfr['Validação Cruzada'].apply(lambda x: 'X' if x == 'k=3' else '')
df_rfr['KFold k=10'] = df_rfr['Validação Cruzada'].apply(lambda x: 'X' if x == 'k=10' else '')
df_rfr['RepeatedKFold 5x2'] = df_rfr['Validação Cruzada'].apply(lambda x: 'X' if x == 'rep=5x2' else '')


### 8. Dataframes finais

In [30]:
# Random Forest
df_rfr = df_rfr[[
    'Cenário', '2 classificações', '3 classificações', 
    'Standard', 'MinMax', 'Robust',
    'Remoção de Outliers',
    '-','RFE','SelectKBest',
    'Critério',
    'Nº Árvores',
    'KFold k=3','KFold k=10','RepeatedKFold 5x2',
    'MSE', 'MAE', 'R²'
]]

# XGBoost
df_xgbr = df_xgbr[[
    'Cenário', '2 classificações', '3 classificações', 
    'Remoção de Outliers', 
    'Standard', 'MinMax', 'Robust',
    'Remoção de Outliers',
    '-','RFE','SelectKBest',
    'Taxa 0.1', 'Taxa 0.3', 
    'Profundidade 3', 'Profundidade 5',
    'KFold k=3','KFold k=10','RepeatedKFold 5x2',
    'MSE', 'MAE', 'R²'
]]

### 6. Exportação

In [31]:
with pd.ExcelWriter('../../../data/plano-de-experimentacao/plano_de_experimentacao_final_2.xlsx', engine='xlsxwriter') as writer:
    df_rfr.to_excel(writer, sheet_name='Random Forest Regressor', index=False)
    df_xgbr.to_excel(writer, sheet_name='XGBoost Regressor', index=False)

print("Excel gerado com sucesso!")

Excel gerado com sucesso!
